In [1]:
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException as SERE
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as BS
import os, time
from shutil import which

In [2]:
def find(driver,idstring):
    element = driver.find_elements_by_id(idstring)
    if element:
        return element
    else:
        return False

In [13]:
PROPERTYLINK = 'http://sdat.dat.maryland.gov/RealProperty/Pages/default.aspx'
COUNTY = "PRINCE GEORGE'S %s" % "COUNTY"
METHOD =  "STREET ADDRESS"
QUERY = "Oakridge"
chromedriver = which('chromedriver')
#chromedriver="/usr/bin/chromedriver"
if chromedriver:
    os.environ["webdriver.chrome.driver"] = chromedriver
else:
    print('chromedriver not found')

In [14]:
driver = webdriver.Chrome(chromedriver)
driver.wait = Wait(driver, 5)
driver.get(PROPERTYLINK)

In [15]:
def ProcessPage1():
cntyID = 'MainContent_MainContent_cphMainContentArea_ucSearchType_wzrdRealPropertySearch_ucSearchType_ddlCounty'
selectCnty = Select(driver.find_element_by_id(cntyID))
selectCnty.select_by_visible_text("PRINCE GEORGE'S COUNTY")
methodID = 'MainContent_MainContent_cphMainContentArea_ucSearchType_wzrdRealPropertySearch_ucSearchType_ddlSearchType'
selectMethod = Select(driver.find_element_by_id(methodID))
selectMethod.select_by_visible_text('STREET ADDRESS')
wait = Wait(driver, 10)
buttonID = 'MainContent_MainContent_cphMainContentArea_ucSearchType_wzrdRealPropertySearch_StartNavigationTemplateContainerID_btnContinue'
trycount = 0
while True:
    trycount += 1
    try:
        print('attempt # %d' % trycount)
        element = wait.until(EC.element_to_be_clickable((By.ID, buttonID)))
        if element:
            element.click()
            break
    except SERE as e:
        print('Element went stale :(')
        print('waiting a bit before trying')
        if trycount > 5:
            break
        time.sleep(2)

attempt # 1
Element went stale :(
waiting a bit before trying


Exception ignored in: <bound method Service.__del__ of <selenium.webdriver.chrome.service.Service object at 0x106b9c780>>
Traceback (most recent call last):
  File "/Applications/anaconda/lib/python3.5/site-packages/selenium/webdriver/common/service.py", line 151, in __del__
    self.stop()
  File "/Applications/anaconda/lib/python3.5/site-packages/selenium/webdriver/common/service.py", line 123, in stop
    if self.process is None:
AttributeError: 'Service' object has no attribute 'process'


attempt # 2


In [7]:
stNmID = 'MainContent_MainContent_cphMainContentArea_ucSearchType_wzrdRealPropertySearch_ucEnterData_txtStreetName'
stNmBx = driver.find_element_by_id(stNmID)
stNmBx.send_keys('Oakridge')
nxtBtnID = 'MainContent_MainContent_cphMainContentArea_ucSearchType_wzrdRealPropertySearch_StepNavigationTemplateContainerID_btnStepNextButton'
driver.implicitly_wait(5)
try:
    element = wait.until(EC.element_to_be_clickable((By.ID, nxtBtnID)))
    if element:
        element.click()
except SERE as e:
    print('Element went stale: (')

In [8]:
pagesource = driver.page_source

In [9]:
soup = BS(pagesource, 'lxml')

In [10]:
#Getting the named_link table of the third page
tbl = soup.find('table',attrs={"class":"ui-table table-stripe"})

In [11]:
tbody = tbl.find('tbody')

In [12]:
linklist=[]
for row in tbody.find_all('tr'):
    a = row.find('a', attrs={"class": "lnkdetails"})
    if a:
        dataLinkEle = driver.find_element(By.ID, a.attrs['id'])
        linklist.append(dataLinkEle)

In [18]:
# clicking on individual names
dataLinkEle.click()

In [20]:
# Page 4:
srcIndiv = driver.page_source

In [22]:
iSoup = BS(srcIndiv,'lxml')

In [25]:
iTbl = iSoup.find('table',attrs={'id':'detailSearch'})

In [26]:
itbody = iTbl.find('tbody')

In [30]:
# THIS SHOULD GIVE ME THE LEVEL-1 ROWS OF THE MAIN TABLE. WE CAN SKIP
trows = itbody.find_all('tr', recursive=True) 

In [31]:
len(trows) # we have to go 

50

In [45]:
row0 = trows[0]
row0.text.strip().replace('\n','') == '' # true so skip

True

In [51]:
for i, row in enumerate(trows):
    text = row.text.strip().replace('\n',' ')
    if text == '':
        print('skip row %d' %i)
    elif 'Information' in text:
        print('-' * 50)
        print('NEW SECTION: %s' % text)
    else:
        print(text)
    print('-'*50)

skip row 0
--------------------------------------------------
Account Identifier:    District - 19 Account Number - 2155224
--------------------------------------------------
--------------------------------------------------
NEW SECTION: Owner Information
--------------------------------------------------
Owner Name:   CORBOY EILEEN    Use:  Principal Residence:    RESIDENTIAL YES
--------------------------------------------------
Mailing Address:   6919 OAKRIDGE RDHYATTSVILLE MD 20782-1153   Deed Reference:    /19907/ 00602
--------------------------------------------------
--------------------------------------------------
NEW SECTION: Location & Structure Information
--------------------------------------------------
Premises Address:    6919  OAKRIDGE RDHYATTSVILLE          20782-0000     Legal Description:    T-DT S/B 07/12/04L19907 F602
--------------------------------------------------
Map:   Grid:   Parcel:   Sub District:   Subdivision:   Section:   Block:   Lot:   Assessment

In [ ]:
def ParseOwnerInfo:
    pass

def ParseLocStrucInfo:
    pass

def ParseValueInfo:
    pass

def ParseTransferInfo:
    pass